In [ ]:
! pip install -U langchain-nomic langchain_community tiktoken langchainhub langchain langgraph tavily-python llama-index llama-parse llama-index-llms-groq llama-index-embeddings-fastembed fastembed  llama-index-vector-stores-qdrant

In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00


#### we are using Davidson's Medicine, a widely referenced book across medical institutions globally as a context to the LLM

In [ ]:
# as we are specifically focusing on cardio-vascular diseases, we extract only that portion from the book and make it a pdf named cardio-vascular.pdf
import PyPDF2

def extract_pages(input_pdf, output_pdf, start_page, end_page):

    with open(input_pdf, 'rb') as infile:
        pdf_reader = PyPDF2.PdfReader(infile)
        pdf_writer = PyPDF2.PdfWriter()

        # Ensure the start and end pages are within the range of available pages
        if start_page < 0 or end_page >= len(pdf_reader.pages) or start_page > end_page:
            raise ValueError("Page range is out of bounds.")

        # Add specified pages to the writer
        for page_num in range(start_page, end_page + 1):
            pdf_writer.add_page(pdf_reader.pages[page_num])

        # Write the output PDF file
        with open(output_pdf, 'wb') as outfile:
            pdf_writer.write(outfile)

# Define the input PDF, output PDF, and page range to extract (0-based index)
input_pdf_path = '/content/drive/MyDrive/Davidson Medicine 24th.pdf'
output_pdf_path = '/content/cardio_vascular.pdf'
start_page = 428
end_page = 523

extract_pages(input_pdf_path, output_pdf_path, start_page, end_page)

In [ ]:
# the context cannot be passed directly to the LLMs because of their limited context window.so we need to parse them into chucks for which we use llamaparse
import os
import nest_asyncio
nest_asyncio.apply()
os.environ['LLAMA_PARSE_API_KEY']="your-llamaparse-api-key"

from llama_parse import LlamaParse
llama_parse_documents = LlamaParse(api_key="your-llamaparse-api-key", result_type="markdown").load_data(["/content/cardio_vascular.pdf","/content/cardiology-explained.pdf"])

Parsing files: 100%|██████████| 2/2 [00:20<00:00, 10.09s/it]


In [2]:
from llama_index.embeddings.fastembed import FastEmbedEmbedding

embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

In [3]:
# groq provides state of the art open source models like llama3 and mixtral.we will be sticking with llama3
import os
from llama_index.llms.groq import Groq

os.environ["GROQ_API_KEY"]='your-groq-api-key'
llm1 = Groq(model="Llama3-70b-8192", api_key="your-groq-api-key")

In [4]:
# by default llamaindex uses OPENAI's llm and embedding model. so we explicitly mention which models are we using (llama3 and FastEmbedding)
from llama_index.core import Settings
Settings.llm=llm1
Settings.embed_model=embed_model

In [5]:
# we will be using qdrant as our vectorstore. one might use chromadb (free) or pinecone databases as well
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
import qdrant_client

client = qdrant_client.QdrantClient(
    "your-qdrant-endpoint", # you can obtain this from qdrant cloud
    api_key="your-qdrant-api-key",  #qdrant api key
)

vector_store = QdrantVectorStore(client=client, collection_name="health_documents")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index =VectorStoreIndex.from_documents(documents=llama_parse_documents,storage_context=storage_context,show_progress=True)
#once vectorstore is created, from next time we don't need to create is again. we can just index it by:

# index = VectorStoreIndex.from_vector_store(vector_store =vector_store)

In [6]:
from IPython.display import display, Markdown

In [43]:
query_engine = index.as_query_engine()
response = query_engine.query("What are the potentially reversible causes of cardiac arrest that should be considered during advanced life-support, and how would you prioritize them in your management approach?")
display(Markdown(response.response))

Based on the provided context, the potentially reversible causes of cardiac arrest that should be considered during advanced life-support are:

1. Hypoxia
2. Hypovolemia
3. Hyper-/hypokalemia and metabolic disorders
4. Hypothermia
5. Tamponade
6. Tension pneumothorax
7. Toxic/therapeutic disorders
8. Thromboembolic and mechanical obstruction

In terms of prioritization, it is essential to address the most critical and life-threatening causes first. Therefore, the management approach should prioritize:

1. Hypoxia and Hypovolemia: Ensure adequate oxygenation and fluid resuscitation to prevent further deterioration.
2. Tamponade and Tension pneumothorax: Address these life-threatening conditions promptly to prevent cardiac tamponade and respiratory failure.
3. Hyper-/hypokalemia and metabolic disorders: Correct any electrolyte imbalances and metabolic disorders that may be contributing to the cardiac arrest.
4. Hypothermia: Manage hypothermia promptly to prevent further complications.
5. Toxic/therapeutic disorders: Identify and manage any toxic or therapeutic disorders that may be contributing to the cardiac arrest.
6. Thromboembolic and mechanical obstruction: Address any thromboembolic or mechanical obstruction that may be contributing to the cardiac arrest.

By prioritizing these reversible causes, healthcare providers can increase the chances of successful resuscitation and improve patient outcomes.

In [29]:
retriever = index.as_retriever(search_kwargs={"k":5})

In [ ]:
os.environ['LANGCHAIN_TRACING-V2']='true'
os.environ['LANGCHAIN_ENDPOINT']='https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY']='your-langchain-api-key'

In [26]:
%pip install -qU langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 1.2 MB/s eta 0:00:00


In [27]:
import time
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser

In [28]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature=0,
    model="Llama3-70b-8192",
    api_key="your-groq-api-key"
)

In [30]:
# generation
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks.Only
    Use the following pieces of retrieved context to answer the question.If context doesnot provide information to answer that question, just say user to do web search.
    If you find it,try to explain the answer keeping the answer concise. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question}
    Context: {context}
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "document"],
)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
start = time.time()
rag_chain = prompt | llm | StrOutputParser()
question = "how genetics causes hypertension."
docs = retriever.retrieve(question)
docs = [d.text for d in docs]
generation = rag_chain.invoke({"context":docs,"question": question})
print(generation)
end = time.time()
print(f"The time required to generate response by Router Chain in seconds:{end - start}")

According to the provided context, genetics play a role in hypertension, but the exact mechanisms are not fully understood. It is mentioned that BP (blood pressure) is a normally distributed, polygenic trait, meaning that multiple genes interact with each other and the environment to influence blood pressure. However, despite much effort, only a few mutations, such as those of the ACE gene, have been convincingly shown to be associated with hypertension.
The time required to generate response by Router Chain in seconds:8.695170402526855


In [31]:
# answer grader
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether an
    answer is useful to resolve a question. Give a  score 'yes' or 'no' to indicate whether the answer is
    useful to resolve a question. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
     <|eot_id|><|start_header_id|>user<|end_header_id|> Here is the answer:
    \n ------- \n
    {generation}
    \n ------- \n
    Here is the question: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "question"],
)
start = time.time()
answer_grader = prompt | llm | JsonOutputParser()
answer_grader_response = answer_grader.invoke({"question": question,"generation": generation})
end = time.time()
print(f"The time required to generate response by the answer grader in seconds:{end - start}")
print(answer_grader_response)

The time required to generate response by the answer grader in seconds:4.046401500701904
{'score': 'no'}


In [32]:
# web search
from langchain_community.tools.tavily_search import TavilySearchResults
os.environ['TAVILY_API_KEY'] = "your-tavily-api-key"
web_search_tool = TavilySearchResults(k=3)

In [33]:
from typing_extensions import TypedDict
from typing import List

### State

class GraphState(TypedDict):
    question : str
    generation : str
    web_search : str
    documents : List[str]
    urls : List[str]

In [34]:
from langchain.schema import Document

In [35]:
def generate(state):
  question=state["question"]
  documents=state["documents"]
  urls=state["urls"]
  urls=[]

  query_engine = index.as_query_engine()
  # generation
  generation = query_engine.query(question)
  generation = generation.response
  print("GENERATING FROM CONTEXT")
  # using answer grader to grade our answer
  score = answer_grader.invoke({"question": question,"generation": generation})
  print(score['score'])
  if score['score']=='yes':
    print(" CONTEXT RESPONSE IS OK")
    return {"documents": documents, "question": question,"generation":generation,"urls":urls}

  else:
    #state["web_search"]=='yes'
    print("DOING WEB SEARCH")
    while score['score']=='no':

      urls=[]
      # web searching
      docs = web_search_tool.invoke({"query": question})
      # print(docs)
      web_results = "\n".join([d["content"] for d in docs])
      urls.extend(d['url'] for d in docs)
      # print(urls)
      web_results = Document(page_content=web_results)
      if documents is not None:
          documents.append(web_results)
      else:
          documents = [web_results]


      # generation
      generation = rag_chain.invoke({"context": documents, "question": question})
      print("GENERATING FROM WEB_SEARCH")
      # grading answer
      score = answer_grader.invoke({"question": question,"generation": generation})
      if score['score']=='yes':
        print("WEBSEARCH RESULT IS OK")
        return {"documents": documents, "question": question,"generation":generation,"urls":urls}
        break


In [36]:
from langgraph.graph import END, StateGraph
workflow = StateGraph(GraphState)

workflow.add_node("generate",generate)

In [37]:
workflow.set_entry_point("generate")

workflow.set_finish_point("generate")

In [38]:
app = workflow.compile()

In [60]:
from pprint import pprint
inputs = {"question":"What are the potentially reversible causes of cardiac arrest that should be considered during advanced life-support, and how would you prioritize them in your management approach?"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
display(Markdown(value["generation"]))

# if the context is fed using web search ,we also print the urls
if value["urls"] !=[]:
  pprint(value["urls"])

GENERATING FROM CONTEXT
yes
 CONTEXT RESPONSE IS OK
'Finished running: generate:'


Based on the provided context, the potentially reversible causes of cardiac arrest that should be considered during advanced life-support are:

1. Hypoxia
2. Hypovolemia
3. Hyper-/hypokalemia and metabolic disorders
4. Hypothermia
5. Tamponade
6. Tension pneumothorax
7. Toxic/therapeutic disorders
8. Thromboembolic and mechanical obstruction

In terms of prioritization, it is essential to address the most critical and life-threatening causes first. Therefore, the management approach should prioritize:

1. Hypoxia and Hypovolemia: Ensure adequate oxygenation and fluid resuscitation to prevent further deterioration.
2. Tamponade and Tension pneumothorax: Address these life-threatening conditions promptly to prevent cardiac tamponade and respiratory failure.
3. Hyper-/hypokalemia and metabolic disorders: Correct any electrolyte imbalances and metabolic disorders that may be contributing to the cardiac arrest.
4. Hypothermia: Manage hypothermia promptly to prevent further complications.
5. Toxic/therapeutic disorders: Identify and manage any toxic or therapeutic disorders that may be contributing to the cardiac arrest.
6. Thromboembolic and mechanical obstruction: Address any thromboembolic or mechanical obstructions that may be contributing to the cardiac arrest.

By prioritizing these reversible causes, healthcare providers can increase the chances of successful resuscitation and improve patient outcomes.